In [2]:
import pandas as pd
donations = pd.read_csv('C://Users/Nameetha/Desktop/Stata/opendata_donations000.gz',escapechar='\\', names=['_donationid', '_projectid', '_donor_acctid', '_cartid', 'donor_city', 'donor_state', 'donor_zip', 'is_teacher_acct', 'donation_timestamp', 'donation_to_project', 'donation_optional_support', 'donation_total', 'donation_included_optional_support', 'payment_method', 'payment_included_acct_credit', 'payment_included_campaign_gift_card', 'payment_included_web_purchased_gift_card', 'payment_was_promo_matched','is_teacher_referred', 'giving_page_id', 'giving_page_type', 'for_honoree', 'thank_you_packet_mailed'])
# Filter the data for respective year from donations data
Total_donations_2015=donations[(donations['donation_timestamp'].apply(lambda x:x[0:4]) == '2015')]

giftcards = pd.read_csv('C://Users/Nameetha/Desktop/Stata/opendata_giftcards000.gz', escapechar='\\', names=['_giftcardid', 'dollar_tier', '_buyer_acctid', 'buyer_city', 'buyer_state', 'buyer_zip', 'date_purchased', '_buyer_cartid', '_recipient_acctid', 'recipient_city', 'recipient_state', 'recipient_zip', 'redeemed', 'date_redeemed', '_redeemed_cartid', 'payment_method', 'payment_included_acct_credit', 'payment_included_campaign_gift_card', 'payment_included_web_purchased_gift_card', 'payment_was_promo_matched'])
#print("Filtering data for 2015...")
giftcards_2015=giftcards[(giftcards['date_purchased'].apply(lambda x:x[0:4]) == '2015')]
giftcards_2015_Test=giftcards_2015.groupby(['_recipient_acctid'])[['_recipient_acctid']].count()
giftcards_2015_Test=giftcards_2015_Test.rename(columns = {'_recipient_acctid':'_recipient_acctid_Count'})
giftcards_2015_Test.reset_index(inplace=True)
all_recipient_acctid=pd.DataFrame()
all_recipient_acctid["_recipient_acctid"]=(giftcards_2015_Test['_recipient_acctid'].where(giftcards_2015_Test['_recipient_acctid_Count'] == 1))
all_recipient_acctid = all_recipient_acctid.dropna(how='any',axis=0) 
Gift_Merge_=pd.merge(all_recipient_acctid,giftcards_2015, how='left',left_on=['_recipient_acctid'],right_on=['_recipient_acctid'])


C:\Users\Nameetha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
Total_donations_2015.payment_included_campaign_gift_card.replace(['t', 'f'], [1, 0], inplace=True)
Total_donations_2015.payment_included_web_purchased_gift_card.replace(['t', 'f'], [1, 0], inplace=True)
Final_Total_donations_2015=pd.DataFrame()
Final_Total_donations_2015=Total_donations_2015.loc[ (Total_donations_2015.payment_included_web_purchased_gift_card==1) |
                                                    (Total_donations_2015.payment_included_web_purchased_gift_card==1)]

C:\Users\Nameetha\Anaconda3\lib\site-packages\pandas\core\generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [5]:
Gift_Donation_Merge=pd.merge(Gift_Merge_,Final_Total_donations_2015, how='inner',left_on=['_recipient_acctid',(Gift_Merge_["date_redeemed"]).astype(str).apply(lambda x:x[0:10])],
                     right_on=['_donor_acctid',(Final_Total_donations_2015["donation_timestamp"]).astype(str).apply(lambda x:x[0:10])])


In [6]:
Gift_Donation_Merge.shape

(18201, 44)

In [10]:
Gift_Donation_Merge.payment_method_y.replace(['no_cash_received', 'promo_code_match', 'creditcard', 'paypal',
       'amazon', 'double_your_impact_match', 'check', 'almost_home_match'], [1,0, 0, 0,
      0, 0, 0, 0], inplace=True)

In [20]:
Gift_Donation_Merge["Top_Up"]= ((Gift_Donation_Merge.payment_included_campaign_gift_card_y==1) |
                                                    (Gift_Donation_Merge.payment_included_web_purchased_gift_card_y==1)) & (Gift_Donation_Merge.payment_method_y==0) 

In [28]:
Gift_Donation_Merge.to_csv('C://Users/Nameetha/Desktop/Stata/Top_UP_2015_For_Logistic_Regression.csv')

In [23]:
Gift_Donation_Merge

,_recipient_acctid,key_1,_giftcardid,dollar_tier,_buyer_acctid,buyer_city,buyer_state,buyer_zip,date_purchased,_buyer_cartid,...,payment_included_acct_credit_y,payment_included_campaign_gift_card_y,payment_included_web_purchased_gift_card_y,payment_was_promo_matched_y,is_teacher_referred,giving_page_id,giving_page_type,for_honoree,thank_you_packet_mailed,Top_Up
0,0000fc11407901bcacdfad1db909b9f6,2015-09-04,1b24d914c61e31aec3a6e12dd1abfc34,100_and_up,9962ffe549afce18ad6676142d370bfa,NaN,AL,357.0,2015-09-04,a41c9ec0cca63a763d521e0b2bc49128,...,f,0,1,f,t,560483c6dc1bbbb321972b655b0d2048,teacher,f,f,False
1,00087bdaa579b108e0b96030a33a0065,2015-09-19,fc8928bb140905490c7965a74c72070a,10_to_100,62cba1f0fe6e073bf077ee8ed654f789,NaN,MN,101.0,2015-09-19,NaN,...,f,0,1,f,f,NaN,NaN,f,f,False
2,000b476cefb172411486ba92e80fbed3,2015-11-11,c0c3a05e8d9080274d33447b2fd0b2a4,10_to_100,cde384c0f20468f64e693b62bf16bf35,Alpharetta,GA,300.0,2015-10-14,37a3e1dade0096862e6585589f58e97a,...,f,0,1,f,f,NaN,NaN,f,f,False
3,000c4bce0a76e3f9e5ebd17615a2483d,2015-11-21,8e346eca2c4b982e31a3f4a5039fc238,100_and_up,10f875467a3d7dc5f247064b4a8e715a,NaN,NY,112.0,2015-11-11,a4b26f39f222a7bf287e8a42dabbb819,...,f,0,1,f,t,c6c108210686919f7bd3507386638e10,teacher,f,f,False
4,000e798165db6e2714d3dd437b239fc2,2015-09-16,6f49859313dfca04ce454006fcba81e8,10_to_100,62cba1f0fe6e073bf077ee8ed654f789,NaN,MN,101.0,2015-09-16,NaN,...,f,0,1,f,f,NaN,NaN,f,f,False
5,000ecd50f38445843c0abe5cb72fe6f5,2015-10-29,c32817b958e63a0d6d9b84c257ef91b0,10_to_100,4e2aa27aaf03274a0c913ffab52e5c76,NaN,IN,469.0,2015-10-29,7d36e595f4e52e79b48f8adad3517a51,...,f,0,1,f,f,NaN,NaN,f,f,False
6,000fc4c68b8650b76b194a78c79267e2,2015-04-07,4293e5ab5c51d7174f1c038ceae05527,10_to_100,13132adb075321191385af89f35547da,NaN,PA,NaN,2015-04-04,NaN,...,f,0,1,f,t,eac0289190f4c6751472017cf1c837a8,teacher,f,f,False
7,0012ef764a10a44c1a51ec970f4ca78e,2015-10-04,d5e73c589fd36e9f9a076d5b13497a49,10_to_100,bca8cbfdb4d48ecf707a9f0fa51a02a9,Chapel Hill,NC,275.0,2015-09-21,8f2e13ab853061171dd239c24784834a,...,f,0,1,f,f,NaN,NaN,f,f,False
8,00147fccd962ca3313447e8a187c4137,2015-12-15,3f6f798d2860be7495c4e7a756a23380,10_to_100,a5d1abd18d9b237047f9c7e666b8a4e6,San Francisco,CA,941.0,2015-12-07,b1efefe03f06eddd9c6b0e85fee096ab,...,f,0,1,f,f,NaN,NaN,f,f,False
9,00199e3565635f8a5ebefd3b5985a7f3,2015-05-29,7ef8a95a0d61193d19ad5c439e560211,100_and_up,00199e3565635f8a5ebefd3b5985a7f3,Newton,MA,24.0,2015-03-25,NaN,...,f,0,1,f,f,NaN,NaN,f,t,True
